In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
input_shape = (28,28,1)

x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test=x_test/255.0

In [ ]:
# y_train = np.array([y_train[i][0] for i in range(len(y_train))])
# y_test = np.array([y_test[i][0] for i in range(len(y_test))])

In [ ]:
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [ ]:
latent_dim = 64 

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(64, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(784, activation='sigmoid'),
      layers.Reshape((28,28,1))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
  
autoencoder = Autoencoder(latent_dim) 

**CNN**

In [ ]:
batch_size = 64
num_classes = 10
epochs = 20

In [ ]:
input_shape = (28,28,1)

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics='accuracy')

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=40)

In [ ]:
saving_dict = {}
saving_dict[10] = history.history

In [ ]:
import pickle
with open('central.txt','wb') as fp:
  pickle.dump(saving_dict,fp)

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

Peer to Peer Learning

In [ ]:
for m in range(2,11,2):
  num_nodes = m
  each_dataset = int(x_train.shape[0]/num_nodes)
  x_train_list = []
  history_list = []
  autoencoder_list = []
  encoded_images_train = []
  decoded_images_train = []
  encoded_images_test = []
  decoded_images_test = []

  for i in range(num_nodes):
    x_train_list.append(x_train[int(i*each_dataset):int((i+1)*each_dataset)])

    autoencoder_list.append(Autoencoder(latent_dim)) 
    autoencoder_list[i].compile(optimizer='adam', loss=losses.MeanSquaredError())

    autoencoder_list[i].fit(x_train_list[i], x_train_list[i],
                  epochs=20,
                  shuffle=True,
                  validation_data=(x_test, x_test))
    
    encoded_images_train.append(autoencoder_list[i].encoder(x_train_list[i]).numpy())
    # decoded_images_train.append(autoencoder_list[i]r.decoder(encoded_images_train[i]).numpy())
    encoded_images_test.append(autoencoder_list[i].encoder(x_test).numpy())
    # decoded_images_test.append(autoencoder_list[i].decoder(encoded_images_test[i]).numpy())

    n = 10
    plt.figure(figsize=(20, 4))
    for j in range(n):
      # display original
        ax = plt.subplot(2, n, j + 1)
        plt.imshow(x_test[j,:,:,0])
        plt.title("original")
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, j + 1 + n)
        plt.imshow(encoded_images_test[i].reshape(encoded_images_test[i].shape[0],8,8,1)[j,:,:,0])
        plt.title("reconstructed")
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()
  overall_encoded_train = np.zeros((x_train.shape[0],8,8,1))

  for k in range(num_nodes):
    overall_encoded_train[int(k*each_dataset):int((k+1)*each_dataset)] = encoded_images_train[k].reshape(encoded_images_train[k].shape[0],8,8,1)

  overall_encoded_train = overall_encoded_train.reshape((overall_encoded_train.shape[0], 8,8, 1))

  model2 = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu', input_shape=(8,8,1)),
      tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu'),
      tf.keras.layers.MaxPool2D(),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
      tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
      tf.keras.layers.MaxPool2D(strides=(2,2)),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(num_classes, activation='softmax')
  ])

  model2.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

  history = model2.fit(overall_encoded_train, y_train,
                      batch_size=batch_size,
                      epochs=20,
                      validation_split=0.1)

  for l in range(num_nodes):
    test_loss, test_acc = model2.evaluate(encoded_images_test[l].reshape(10000,8,8,1), y_test)
  saving_dict[m] = history.history

In [ ]:
import pickle
with open('ae.txt','wb') as fp:
  pickle.dump(saving_dict,fp)